## Simple MNL modelling on the SwissMetro dataset

We use the Swissmetro dataset to compare the implementations of the simple MNL and the reslogit model [1]. 

In [ ]:
import os

# Remove/Add GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


import sys

sys.path.append("../../")

import timeit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from choice_learn.data import ChoiceDataset
from choice_learn.models.reslogit import ResLogit
from choice_learn.models.simple_mnl import SimpleMNL
from choice_learn.datasets import load_swissmetro

Firs, we create a ChoiceDataset from the dataframe.

In [ ]:
dataset = load_swissmetro(as_frame=False) # Use a preprocessing?

In [ ]:
dataset.summary()
print(f"\n\n{type(dataset)=}")
print(f"\n{np.shape(dataset.items_features_by_choice)=}")
print(f"{np.shape(dataset.shared_features_by_choice)=}")

We just use a numpy based train/test split for cross-validation, but the core code is the same!

In [ ]:
n_items = np.shape(dataset.items_features_by_choice)[2]
n_items_features = np.shape(dataset.items_features_by_choice)[3]
n_shared_features = np.shape(dataset.shared_features_by_choice)[2]
n_vars = n_items_features + n_shared_features
n_choices = len(np.unique(dataset.choices))
print(f"{n_items=}\n{n_items_features=}\n{n_shared_features=}\n{n_vars, n_choices=}\n\n")

In [ ]:
indexes = np.random.permutation(list(range(len(dataset))))

fit_losses = []
test_eval = []
for i in range(5):
    start_time = timeit.default_timer()

    test_indexes = indexes[int(len(indexes) * 0.2 * i):int(len(indexes) * 0.2 * (i + 1))]
    train_indexes = np.concatenate([indexes[:int(len(indexes) * 0.2 * i)],
                                    indexes[int(len(indexes) * 0.2 * (i + 1)):]],
                                    axis=0)
    
    # Ensure indexes are not empty
    if len(train_indexes) == 0 or len(test_indexes) == 0:
        print(f"Skipping iteration {i} due to empty indexes.")
        continue

    train_dataset = dataset[train_indexes]
    test_dataset = dataset[test_indexes]

    model = SimpleMNL()
    model.instantiate(n_items=n_items, n_shared_features=n_shared_features, n_items_features=n_items_features)

    losses = model.fit(choice_dataset=train_dataset)
    probas = model.predict_probas(test_dataset)

    eval = tf.keras.losses.CategoricalCrossentropy(from_logits=False)(y_pred=probas, y_true=tf.one_hot(test_dataset.choices, n_items))
    test_eval.append(eval)
    print(test_eval)

    fit_losses.append(losses)

    end_time = timeit.default_timer()
    print(f"Execution time for iteration {i}: {end_time - start_time} seconds")

In [ ]:
cmap = plt.cm.coolwarm
colors = [cmap(j / 4) for j in range(5)]
for i in range(len(fit_losses)):
    plt.plot(fit_losses[i]["train_loss"], label=f"Training (fold {i})", c=colors[i], linestyle="--")
    #plt.plot(fit_losses[i]["test_loss"], label=f"Prediction (fold {i})", c=colors[i])
plt.xlabel("Epochs")
plt.ylabel("Negative log likelihood")
plt.title("MNL model")
plt.legend()

In [ ]:
model.evaluate(test_dataset)

In [ ]:
print("Average LogLikeliHood on test:", np.mean(test_eval))

### References
[1] ResLogit: A residual neural network logit model for data-driven choice modelling, Wong, M.; Farooq, B (2021), Transportation Research Part C: Emerging Technologies 126\
(URL: https://doi.org/10.1016/j.trc.2021.103050)